# *Elements of Programming Interviews* solution notes
This notebook will hold my attempts at problems from *Elements of Programming Interviews* in Python. For each problem, record
* Brute force solution code
* Optimized solution code
* Book solution code
* Comments
    1. How does my solution compare with the book solution?
    2. What went right in my attempt?
    3. What went wrong in my attempt
    4. Do I need to learn new data structure or algorithms?

## Primitive Types

## Arrays 

### The Dutch national flag problem

### Increment an arbitrary precision integer

### Multiply two arbitrary precision integer

### Advancing through an array

### Delete duplicates from a sorted array

### Buy and sell a stock once 
It was helpful to think of the prices as elevation, with the array as a linear trail consisting of a series of peaks and valleys. I am a climber who would like to know the greatest increase in elevation while traveling one direction along the array. I can teleport, but I don't know the elevation at a given point until I travel to that location. I am given two flags labeled `lo` and `hi`.  

I begin by travelling downhill until I reach a valley as a starting point. If I reach the end of the trail without finding a valley, it means I never increased elevation and should return `0`. After placing `lo` at my starting point, I travel along the entire trail to find the highest peak and place `hi` at this location. I then move `lo` forward to the lowest valley before `hi` and record the elevation gain from `lo` to `hi` in my notebook. From this point, there could be a larger elevation change after `hi`, but not before it. I pick up my flags and repeat my steps on the remaining section of the trail, marking down the new elevation change. I will eventually reach the end of the trail, at which point I return the greatest elevation change I recorded in my notebook.

Consider a trail with $n$ points. The fastest time is if there is no elevation gain, in which case it takes $n-1$ steps to reach the end of the trail. If the highest peak is at the very end of the trail, the slowest time will be if the lowest valley is just before the highest peak at $n-2$. It will take $n-1$ steps to find the peak and $n-2$ steps to find the trail, or $2n-3$ steps in total. The worst case time will be $\sum_{i=1}^{m}(2l_i - 3)$, where $m$ is the number of segments and $l_i$ is the length of the $i^{th}$ segment. Thus, the time complexity is $O(n)$.

In [2]:
def buy_and_sell_stock_once(prices):
    n = len(prices)
    max_profit = 0
    lo = 0
    while lo < n-1:
        # Move lo to valley as starting point
        while prices[lo+1] <= prices[lo]:
            lo += 1
            if lo == n-1:
                return max_profit
        # Move hi to highest peak
        hi = lo+1
        for i in range(lo+1, n):
            if prices[i] > prices[hi]:
                hi = i
        # Move lo to lowest valley before hi 
        for i in range(lo+1, hi):
            if prices[i] < prices[lo]:
                lo = i
        # Record elevation gain
        local_profit = prices[hi] - prices[lo]
        max_profit = max(max_profit, local_profit)
        # Check next sub-array
        lo = hi + 1
    return max_profit

1. My solution works, but is much less compact and readable.
2. I solved the problem with brute force method quickly, and ended up with a working faster solution.
3. I started coding too quickly and missed the more compact solution. Thus the total time I took to solve the problem was much too long. 
4. Nothing new to learn.

### Buy and sell a stock twice
After finding the $O(n^2)$ solution of running the one buy/sell code on the left and right parts of the array, I read the description of the optimal solution but didn't copy; I tried to implement it myself. It basically goes through each day and looks at the maximum profit one could make by buying on a previous day selling today, as well as buying today and selling on a future day. The sum of these values is the maximum profit on a given day. The time complexity is $O(n)$ and the space complexity is $O(n)$.

In [24]:
def buy_and_sell_stock_twice(prices):
    min_so_far, max_so_far = float('inf'), float('-inf')
    n = len(prices)
    L, R = [0] * n, [0] * n
    for i in range(1, n):
        j = n-1-i
        min_so_far = min(min_so_far, prices[i-1])
        max_so_far = max(max_so_far, prices[j+1])
        L[i] = max(L[i-1], prices[i] - min_so_far)
        R[j] = max(R[j+1], max_so_far - prices[j])
    return max([sell+buy for sell,buy in zip(L, R)])

1. The book solution uses enumerate and is prettier. I guess the variable names are also more meaningful.
2. I got the $O(n^2)$ solution right away. Once I read the approach to the $O(n)$ solution, I was able to implement it myself.
3. I had trouble coming up with the $O(n)$ solution. 
4. Nothing new to learn, but be comfortable using `enumerate` and `reverse` to iterate backwards through lists.

### Rearrange an array 
I realized I can just sort locally based on hint from book.

In [25]:
def rearrange(A):
    for i in range(len(A)-1): 
        if (i % 2 and A[i] > A[i+1]) or (not i % 2 and A[i] < A[i+1]):
            A[i], A[i+1] = A[i+1], A[i]

1. My solution is harder to read than the book solution.
2. I got an initial solution down quickly.
3. I got hung up on finding the median, which is not super easy. This distracted me from the other $O(n)$ solution.
4. Nothing new to learn.

### Enumerate all primes to n 
I've done this problem before. Starting from i=2, run through the numbers and eliminate all multiples of i. Then move to the next number that's not crossed out and repeat. Do this until i >= $\sqrt{n}$. Here is the book's method. The next iteration does some optimization, but I won't list it here.

In [26]:
def generate_primes(n):
    primes = []
    is_prime = [False, False] + [True] * (n-1)
    for p in range(2, n+1):
        if is_prime[p]:
            primes.append(p)
            # Remove all multiples of p
            for i in range(p, n+1, p):
                is_prime[i] = False
    return primes

### Permute the elements of an array
$O(n)$ time, $O(n)$ space.

In [27]:
def apply_permutation(perm, A):
    B = [0] * len(A)
    for i in range(len(A)):
        B[perm[i]] = A[i]
    A[:] = B

1. My solution matches the solution in the code, but in the older book I have there is a much more complex solution.
2. This was a pretty easy problem, but I figured out the easy and the optimized solution.
3. I probably took too long.
4. Nothing new to learn.

### Compute the next permutation 
If P[-1] > P[-2], the answer is simply to swap those elements. Otherwise, we need to traverse the array backwards until we find P[i] < P[i+1]. Then find the smallest element in P[i+1:] which is greater than P[i] and swap that element with P[i]. Finally, sort the remaining subarray P[i+1:]. Took around 1 hour to solve.

In [28]:
def next_permutation(perm):
    
    # Check for short arrays
    n = len(perm)
    if n < 2:
        return []
    # Try to swap last two elements
    if perm[-1] > perm[-2]:
        perm[-1], perm[-2] = perm[-2], perm[-1]
    else:
        # Go back until we find an index to be incremented
        i = n - 2
        while perm[i] >= perm[i + 1]:
            i -= 1
            if i < 0: # Last dictionary sorted permutation
                return []
        # Find smallest element in perm[i+1] which is larger than perm[i]
        i_next_largest = i + 1
        for j in range(i + 2, n):
            if perm[j] > perm[i] and perm[j] < perm[i_next_largest]:
                i_next_largest = j
        # Move that element and sort the remaining array
        perm[i], perm[i_next_largest] = perm[i_next_largest], perm[i]
        perm[i+1:] = sorted(perm[i+1:])
    return perm

1. My algorithm is the same as the book solution and works, but is redundant. I don't need to check the case where the array has length 1, and I only need to swap once. It was a quick fix to edit my code after seeing this. 
2. I figured out the solution to the problem by using concrete examples, and I didn't give up.
3. The answer took me over an hour to produce.
4. Nothing new to learn.

### Sample offline data

In [9]:
def random_sampling(k, A):
    import random
    for i in range(k):
        r = random.randint(i, len(A)-1)
        A[i], A[r] = A[r], A[i]

### Sample online data

### Compute a random permutation
Start with an array $A$ with $A[i] = i$ for i in range 0 to n-1, inclusive. Choose random index 0 to n-1 and swap this with the first element. This first element is then uniformly random. Now move on to the second element and choose a random index 1 to n-1, then swap with that element. Repeat for the entire array.

In [30]:
import random

def compute_random_permutation(n):
    
    A = list(range(n))
    for i in range(n):
        r = random.randint(i, n-1)
        A[i], A[r] = A[r], A[i]
    return A

1. I like my solution better. The book does something similar but it's explained in a weird way.
2. I solved the problem quickly. It wasn't too hard.
3. Nothing went wrong.
4. Nothing to learn.

### Compute a random subset

### Generate nonuniform random numbers
In the case of continuous variables, the cumulative distribution function $F$ is defined as $F\left(x\right) = \int_{-\infty}^{x} p\left(x{'}\right)dx{'}$. Inverse sampling can be used to generate the desired distribution, in other words finding $x\left(F\right)$. 

Another way to view this: there is an array with $k$ bins, and we choose a random position along the array. If we land inside a bin, we return the number in the bin. Inverse sampling will change the bin sizes according to their probabilities. 

This will take $O(n)$ time to seach the array $F$ for the correct position of the generated random number. 

In [31]:
def nonuniform_random_number_generation(values, probabilities):
    # Find cumulative probabilities
    F = list(itertools.accumulate(probabilities))
    # Choose random element
    r = random.random()
    # Search for correct position in F
    for i in range(len(F)):
        if r <= F[i]:
            return values[i]

1. Book solution efficiently use *bisect* for binary search. 
2. I had the right idea for the solution. 
3. I didn't make the connection between what I knew was the correct method and how to put that into code. I also took too long.
4. Couldn't hurt to learn all the standard modules.

## Strings

### Interconvert strings and integers 

In [32]:
def int_to_string(x: int) -> str:

    negative = x < 0
    x = abs(x)

    s = ''
    while True:
        s = ''.join([chr(ord('0') + x % 10), s])
        x = x // 10
        if x == 0:
            break

    if negative:
        s = ''.join(['-', s])
    return s


def string_to_int(s: str) -> int:

    x, sign = 0, +1
    str_to_int_dict = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4,
                       '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}
    if s[0] not in str_to_int_dict:
        sign = 1 - 2*(s[0] == '-')
        s = s[1:]
    for i, char in enumerate(reversed(s)):
        x += (10**i) * str_to_int_dict[char]
    return sign * x

### Base conversion
Find X, the number in base 10. Then convert to base b2 by a series of mods and divides.

In [1]:
def convert_base(num_as_string: str, b1: int, b2: int) -> str:

    # Strip leading +/- sign
    is_negative = False
    if num_as_string[0] in ['+', '-']:
        is_negative = num_as_string[0] == '-'
        num_as_string = num_as_string[1:]

    # Convert from base b1 to base 10
    X, factor = 0, 1
    for i, char in enumerate(reversed(num_as_string)):
        number = ord(char) - ord('0')
        if number > 9:
            number = (ord(char) - ord('A') + 10)
        X += int(number) * factor
        factor *= b1

    # Convert to X to base b2
    start_index = int(math.log(X, b2)) if X > 0 else 0
    A = [0] * (start_index + 1)
    factor = b2 ** start_index
    for i in range(len(A)):
        A[i] = int(X // factor)
        X %= factor
        factor /= b2
        if X == 0:
            break

    # Change numbers larger than 10 to letters
    for i, number in enumerate(A):
        if number >= 10:
            A[i] = chr(ord('A') - 10 + number)

    # Convert list to string representation
    num_as_string_base_b2 = ''.join([str(c) for c in A])
    if is_negative:
        num_as_string_base_b2 = ''.join(['-', num_as_string_base_b2])
    return num_as_string_base_b2

### Compute the spreadsheet encoding
Each character in the string has 26 possibilities, so a string with k characters has $26^k$ possibilities. If the second letter is 'B', we know $2 * 26^(2-1)$ combinations came before: 'A', ... , 'Z', 'AA', ... , 'AZ'. Thus for each letter at index i in the reversed string, we add $26**i$ multiplied by the value of the letter to the sum.

In [2]:
def ss_decode_col_id(col: str) -> int:
    n = 0
    for i, letter in enumerate(reversed(col)):
        n += (26**i) * (ord(letter) - ord('A') + 1)
    return n

1. Book solution uses functools.reduce nicely.
2. I solved the problem relatively quickly.
3. I didn't optimize the solution.
4. Learn the standard libraries like functools and itertools. 

### Replace and remove

#### Attempt 1 (almost works)
Move forward through list. If we find an 'a', find a proceeding 'b' and move 'b' to the spot just after 'a' by swapping. Then change both of them to 'd'. Do this again but in reverse. Then clean up any remaining a's or b's. 

In [57]:
def replace_and_remove(size, s):

    # Forward pass
    for i in range(len(s)):
        if s[i] == 'a':
            for j in range(i + 1, size):
                if s[j] == 'b':
                    s[i] = s[j] = 'd'
                    for k in reversed(range(i + 1, j)):
                        s[k], s[k - 1] = s[k - 1], s[k]
                    break
    # Backward pass
    for i in reversed(range(len(s))):
        if s[i] == 'a':
            for j in reversed(range(0, i)):
                if s[j] == 'b':
                    s[i] = s[j] = 'd'
                    for k in range(j, i):
                        s[k], s[k + 1] = s[k + 1], s[k]
                    break
    # Clean up
    current_size, i = size, 0
    while i < current_size:
        if s[i] == 'a':
            s[i] = 'd'
            s.insert(i+1, 'd')
            current_size += 1
        elif s[i] == 'b':
            s.pop(i)
            current_size -= 1
        else:
            i += 1

    return current_size

#### Attempt 2 (used hint)

In [108]:
def replace_and_remove(size, s):
    
    # Remove b's
    i = 0
    while i < len(s):
        if s[i] == 'b':
            
    for i in range(size):
        while s[i] == 'b':
            for j in range(i, size):
                s[j] = s[j + 1]

    # Convert a's to d's
    n_valid = len(s) - s.count('')
    size = n_valid + s.count('a')
    write_idx = size - 1
    for char in reversed(s[:n_valid]):
        if char == 'a':
            s[write_idx] = s[write_idx - 1] = 'd'
            write_idx -= 2
        else:
            s[write_idx] = char
            write_idx -= 1

    return size

1. Book solution cleverly does everything in place.
2. Figured out a solution.
3. Way too long to implement.
4. Nothng new to learn